Bank Marketing（bank-full）数据集 BTTWD 实验

本 notebook 按步骤运行：环境准备 → 加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD 实验 → 桶级分析。



In [1]:
# 步骤0：环境与路径设置
import os, sys
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_holdout_experiment,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = os.path.join(root_path, 'configs', 'bank_bttwd.yaml')
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')



【INFO】【2025-11-25 22:20:48】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\bank_bttwd.yaml
【INFO】【2025-11-25 22:20:55】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')



【INFO】【2025-11-25 22:20:55】【配置-数据】数据集=bank_full, k折=5, 目标列=y, 正类="yes"
【INFO】【2025-11-25 22:20:55】【配置-BTTWD】阈值模式=None, 全局模型=xgb, 桶内模型=none, 后验估计器(兼容字段)=logreg
【INFO】【2025-11-25 22:20:55】【配置-基线】LogReg启用=True, RandomForest启用=False, KNN启用=False, XGBoost启用=True
【INFO】【2025-11-25 22:20:55】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw, target_col_model = load_dataset(cfg)  # 返回用于建模的标签列，例如 "y"

display(df_raw.head())
print('用于建模的标签列:', target_col_model)

# 1）画 0/1 标签比例
class_counts = df_raw[target_col_model].value_counts(normalize=True).sort_index()
ax = class_counts.plot(kind='bar', title='订购 vs 未订购比例')
plt.ylabel('比例')

fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bank_class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()

log_info('【步骤2摘要】Bank Marketing 数据加载与标签分布完成。')



【INFO】【2025-11-25 22:20:56】【数据加载】文本表格 ..\data\bank\bank-full.csv 已读取，样本数=45211，列数=17
【INFO】【2025-11-25 22:20:56】【数据加载】银行营销数据集已读取，标签已映射为0/1，样本数=45211，正类比例=11.70%
【INFO】【2025-11-25 22:20:56】【数据集信息】名称=bank_full，样本数=45211，目标列=y，正类比例=11.70%


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0


用于建模的标签列: y
【INFO】【2025-11-25 22:20:56】【步骤2摘要】Bank Marketing 数据加载与标签分布完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")



【INFO】【2025-11-25 22:20:56】【预处理】连续特征=7个，类别特征=9个
【INFO】【2025-11-25 22:20:56】【预处理】编码后维度=42
【INFO】【2025-11-25 22:20:56】【预处理】编码特征维度=42，样本数=45211
【INFO】【2025-11-25 22:20:56】【步骤3摘要】特征预处理完成：连续=7，类别=9，编码维度=42。


In [5]:
# 步骤4：构建桶树并检查划分
feature_df_for_bucket = df_raw.drop(columns=[target_col_model])
bucket_tree = BucketTree(cfg['BTTWD']['bucket_levels'], feature_names=feature_df_for_bucket.columns.tolist())
bucket_ids_full = bucket_tree.assign_buckets(feature_df_for_bucket)
group_df = df_raw.groupby(bucket_ids_full)[target_col_model].agg(['size', 'mean']).reset_index()
group_df.columns = ['bucket_id', 'count', 'pos_rate']
bucket_df = group_df.sort_values('count', ascending=False).reset_index(drop=True)

display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')




【INFO】【2025-11-25 22:20:56】【桶树】已为样本生成桶ID，共 87 个组合


,bucket_id,count,pos_rate
0,L1_age=30-40|L2_job=white_collar|L3_contact=ce...,7237,0.141219
1,L1_age=40-50|L2_job=white_collar|L3_contact=ce...,3747,0.133974
2,L1_age=30-40|L2_job=blue_collar|L3_contact=cel...,3380,0.099704
3,L1_age=<=30|L2_job=white_collar|L3_contact=cel...,2434,0.199671
4,L1_age=30-40|L2_job=blue_collar|L3_contact=OTHER,2399,0.036682


【INFO】【2025-11-25 22:20:58】【步骤4摘要】桶树划分完成，共有 87 个叶子桶。


In [6]:
# 步骤5：运行基线模型实验占位
# 基线部分在 run_kfold_experiments 内统一调度（仅在 use_kfold=True 时执行）
log_info('【步骤5】基线模型（LogReg / XGBoost）将在交叉验证模式中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')



【INFO】【2025-11-25 22:20:58】【步骤5】基线模型（LogReg / XGBoost）将在交叉验证模式中一并运行。
【INFO】【2025-11-25 22:20:58】【步骤5摘要】基线模型性能将作为后续对比基准。


In [8]:
# 步骤6：运行 BTTWD 实验（k 折或单次留出）
use_kfold_raw = cfg.get('DATA', {}).get('use_kfold', False)
if isinstance(use_kfold_raw, str):
    use_kfold = use_kfold_raw.strip().lower() in ['true', '1', 'yes']
else:
    use_kfold = bool(use_kfold_raw)

if use_kfold:
    log_info('【步骤6】检测到 use_kfold=True，进入 k 折实验。')
    results = run_kfold_experiments(X, y, feature_df_for_bucket, cfg)
    summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
    display(summary_df)
    summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
    fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
    plt.savefig(fig_compare, bbox_inches='tight')
    plt.close()
    log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')
else:
    log_info('【步骤6】use_kfold=False，执行单次留出验证流程。')
    holdout_metrics = run_holdout_experiment(X, y, feature_df_for_bucket, cfg)
    display(pd.DataFrame(holdout_metrics))
    log_info('【步骤6摘要】单次留出验证完成，指标已列出。')



【INFO】【2025-11-25 22:21:40】【步骤6】检测到 use_kfold=True，进入 k 折实验。
【INFO】【2025-11-25 22:21:40】【基线-LogReg】使用决策阈值=0.400（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:103: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:21:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-25 22:21:41】【基线-LogReg】整体指标：AUC_mean=0.907, AUC_std=0.005, BAC_mean=0.701, BAC_std=0.006, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.513, F1_std=0.008, Kappa_mean=0.461, Kappa_std=0.008, MCC_mean=0.471, MCC_std=0.008, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.624, Precision_std=0.018, Recall_mean=0.437, Recall_std=0.015, Regret_mean=0.229, Regret_std=0.003
【INFO】【2025-11-25 22:21:41】【基线-XGB】使用决策阈值=0.400（fixed 模式）


C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:21:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:21:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:21:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.



【INFO】【2025-11-25 22:21:45】【基线-XGB】整体指标：AUC_mean=0.935, AUC_std=0.003, BAC_mean=0.776, BAC_std=0.007, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.605, F1_std=0.010, Kappa_mean=0.553, Kappa_std=0.011, MCC_mean=0.553, MCC_std=0.011, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.606, Precision_std=0.011, Recall_mean=0.604, Recall_std=0.013, Regret_mean=0.185, Regret_std=0.005
【INFO】【2025-11-25 22:21:45】【K折实验】正在执行第 1/5 折...
【INFO】【2025-11-25 22:21:45】【桶树】已为样本生成桶ID，共 87 个组合
【INFO】【2025-11-25 22:21:46】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-25 22:21:46] 桶 L1_age=30-40 子桶样本不足（最小子桶 n=24 < 200），不再细分
[INFO][BT][2025-11-25 22:21:46] 桶 L1_age=40-50 子桶样本不足（最小子桶 n=81 < 200），不再细分
[INFO][BT][2025-11-25 22:21:46] 桶 L1_age=50-60 子桶样本不足（最小子桶 n=74 < 200），不再细分
[INFO][BT][2025-11-25 22:21:46] 桶 L1_age=<=30 子桶样本不足（最小子桶 n=3 < 200），不再细分
[INFO][BT][2025-11-25 22:21:46] 桶 L1_age=>60 子桶样本不足（最小子桶 n=7 < 200），不再细分
【INFO】【2025-11-25 22:21:46】【BTTWD】bucket_estimator=none：不训练桶内局部模型，仅使用全局模型概率做

...

【INFO】【2025-11-25 22:21:50】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-25 22:21:51] 桶 L1_age=30-40 子桶样本不足（最小子桶 n=32 < 200），不再细分
[INFO][BT][2025-11-25 22:21:51] 桶 L1_age=40-50 子桶样本不足（最小子桶 n=81 < 200），不再细分
[INFO][BT][2025-11-25 22:21:51] 桶 L1_age=50-60 子桶样本不足（最小子桶 n=81 < 200），不再细分
[INFO][BT][2025-11-25 22:21:51] 桶 L1_age=<=30 子桶样本不足（最小子桶 n=4 < 200），不再细分
[INFO][BT][2025-11-25 22:21:51] 桶 L1_age=>60 子桶样本不足（最小子桶 n=9 < 200），不再细分
【INFO】【2025-11-25 22:21:51】【BTTWD】bucket_estimator=none：不训练桶内局部模型，仅使用全局模型概率做桶内阈值搜索
【INFO】【2025-11-25 22:21:51】【BTTWD】共生成 86 个叶子桶，其中有效桶 0 个（样本数 ≥ 200）
【INFO】【2025-11-25 22:21:51】【桶树】已为样本生成桶ID，共 79 个组合
【INFO】【2025-11-25 22:21:52】【桶树】已为样本生成桶ID，共 79 个组合
【INFO】【2025-11-25 22:21:52】【桶树】已为样本生成桶ID，共 79 个组合
【INFO】【2025-11-25 22:21:52】【BTTWD】三支指标(含后悔)：Precision=0.542, Recall=0.778, F1=0.639, BAC=0.845, AUC=0.937, MCC=0.594, Kappa=0.581, BND_ratio=0.139, POS_Coverage=0.055, Regret=0.200
【INFO】【2025-11-25 22:21:52】【桶树】已为样本生成桶ID，共 79 个组合
【INFO】【2025-11-25 22:21:52】【K折实验】正在执行第 5/5 折

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:21:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-25 22:21:52】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-25 22:21:53] 桶 L1_age=30-40 子桶样本不足（最小子桶 n=32 < 200），不再细分
[INFO][BT][2025-11-25 22:21:53] 桶 L1_age=40-50 子桶样本不足（最小子桶 n=82 < 200），不再细分
[INFO][BT][2025-11-25 22:21:53] 桶 L1_age=50-60 子桶样本不足（最小子桶 n=70 < 200），不再细分
[INFO][BT][2025-11-25 22:21:53] 桶 L1_age=<=30 子桶样本不足（最小子桶 n=4 < 200），不再细分
[INFO][BT][2025-11-25 22:21:53] 桶 L1_age=>60 子桶样本不足（最小子桶 n=8 < 200），不再细分
【INFO】【2025-11-25 22:21:53】【BTTWD】bucket_estimator=none：不训练桶内局部模型，仅使用全局模型概率做桶内阈值搜索
【INFO】【2025-11-25 22:21:53】【BTTWD】共生成 87 个叶子桶，其中有效桶 0 个（样本数 ≥ 200）
【INFO】【2025-11-25 22:21:53】【桶树】已为样本生成桶ID，共 79 个组合
【INFO】【2025-11-25 22:21:53】【桶树】已为样本生成桶ID，共 79 个组合
【INFO】【2025-11-25 22:21:53】【桶树】已为样本生成桶ID，共 79 个组合
【INFO】【2025-11-25 22:21:53】【BTTWD】三支指标(含后悔)：Precision=0.537, Recall=0.753, F1=0.627, BAC=0.834, AUC=0.928, MCC=0.579, Kappa=0.568, BND_ratio=0.123, POS_Coverage=0.062, Regret=0.205
【INFO】【2025-11-25 22:21:53】【桶树】已为样本生成桶ID，共 79 个组合
【INFO】【2025-11-25 22:21:53】【K折实验】所有结果已写入 res

,model,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,BAC_mean,BAC_std,AUC_mean,...,MCC_mean,MCC_std,Kappa_mean,Kappa_std,BND_ratio_mean,BND_ratio_std,POS_Coverage_mean,POS_Coverage_std,Regret_mean,Regret_std
0,BTTWD,0.533359,0.007349,0.756094,0.013704,0.625463,0.008734,0.834224,0.006912,0.932804,...,0.577625,0.010483,0.565913,0.010023,0.128,0.006261,0.059278,0.00297,0.204475,0.003000
1,LogReg,0.623769,0.018441,0.436565,0.014859,0.513241,0.008340,0.700773,0.006167,0.906632,...,0.470631,0.008159,0.461357,0.008428,0.000,0.000000,NaN,NaN,0.228661,0.003360
2,XGBoost,0.606381,0.010677,0.603890,0.013179,0.605067,0.010262,0.775969,0.006619,0.935239,...,0.552913,0.011429,0.552873,0.011455,0.000,0.000000,NaN,NaN,0.184889,0.004848


【INFO】【2025-11-25 22:21:54】【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。


In [9]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标（如存在）已整理，可用于局部化分析。')



,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,threshold_n_samples,n_all,pos_rate_all,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold,pos_rate
0,L1_age=30-40,L1,NaN,9897,4276,0.102455,0.099392,0.6,0.2,0.176099,...,4276,14173,0.101531,NaN,NaN,NaN,NaN,NaN,1,0.101531
1,L1_age=40-50,L1,NaN,6204,2739,0.089781,0.090544,0.6,0.2,0.166119,...,2739,8943,0.090015,NaN,NaN,NaN,NaN,NaN,1,0.090015
2,L1_age=50-60,L1,NaN,4568,1839,0.099825,0.098423,0.6,0.2,0.175639,...,1839,6407,0.099423,NaN,NaN,NaN,NaN,NaN,1,0.099423
3,L1_age=<=30,L1,NaN,3970,1705,0.159950,0.174780,0.6,0.2,0.294428,...,1705,5675,0.164405,NaN,NaN,NaN,NaN,NaN,1,0.164405
4,L1_age=>60,L1,NaN,678,292,0.442478,0.400685,0.6,0.3,0.542808,...,292,970,0.429897,NaN,NaN,NaN,NaN,NaN,1,0.429897


【INFO】【2025-11-25 22:21:54】【步骤7摘要】桶级指标（如存在）已整理，可用于局部化分析。


In [10]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
results_dir = os.path.join(root_path, cfg['OUTPUT']['results_dir'])
figs_dir = os.path.join(root_path, cfg['OUTPUT']['figs_dir'])
os.makedirs(results_dir, exist_ok=True)
os.makedirs(figs_dir, exist_ok=True)
print(os.listdir(results_dir))
print(os.listdir(figs_dir))
log_info('【全部步骤完成】Bank Marketing 数据集的 BT-TWD 实验结束。')



【INFO】【2025-11-25 22:21:54】【步骤8】检查结果文件与图表。
['bucket_metrics.csv', 'bucket_thresholds_per_fold.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv']
['bank_class_distribution.png', 'bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-11-25 22:21:54】【全部步骤完成】Bank Marketing 数据集的 BT-TWD 实验结束。
